In [1]:
import os
from dotenv import load_dotenv

In [2]:
try:
    load_dotenv()
    print("Load successful")
except Exception as e:
    print(f"Error importing the key {e}")

Load successful


# 🚀 Agent Tool Patterns and Best Practices

**Welcome to Day-2 of the Kaggle 5-day Agents course!**

In the previous notebook, you learned how to add custom Python functions as tools to your agent. In this notebook, we'll take the next step: **consuming external MCP services** and handling **long-running operations**.

In this notebook, you'll learn how to:

- ✅ **Connect to external MCP servers**
- ✅ **Implement long-running operations** that can pause agent execution for external input
- ✅ **Build resumable workflows** that maintain state across conversation breaks
- ✅ Understand when and how to use these patterns

In [4]:
try:
    GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Google Key setup complete")
except Exception as e:
    print(f"Error importing the key and setup {e}")

Google Key setup complete


In [5]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [6]:
retry_config = types.HttpRetryOptions(
        attempts=5,
        exp_base=7,
        initial_delay=1,
        http_status_codes=[429,500,503,504]
)

---
## 🧰 Section 2: Model Context Protocol

So far, you have learned how to create custom functions for your agents. But connecting to external systems (GitHub, databases, Slack) requires writing and maintaining API clients.

**Model Context Protocol (MCP)** is an open standard that lets agents use community-built integrations. Instead of writing your own integrations and API clients, just connect to an existing MCP server.

MCP enables agents to:

✅ **Access live, external data** from databases, APIs, and services without custom integration code  
✅ **Leverage community-built tools** with standardized interfaces  
✅ **Scale capabilities** by connecting to multiple specialized servers

### 2.1: How MCP Works

MCP connects your agent (the **client**) to external **MCP servers** that provide tools:

- **MCP Server**: Provides specific tools (like image generation, database access)
- **MCP Client**: Your agent that uses those tools
- **All servers work the same way** - standardized interface

**Architecture:**
```
┌──────────────────┐
│   Your Agent     │
│   (MCP Client)   │
└────────┬─────────┘
         │
         │ Standard MCP Protocol
         │
    ┌────┴────┬────────┬────────┐
    │         │        │        │
    ▼         ▼        ▼        ▼
┌────────┐ ┌─────┐ ┌──────┐ ┌─────┐
│ GitHub │ │Slack│ │ Maps │ │ ... │
│ Server │ │ MCP │ │ MCP  │ │     │
└────────┘ └─────┘ └──────┘ └─────┘
```

### 2.2: Using MCP with Your Agent

The workflow is simple:

1. Choose an MCP Server and tool
2. Create the MCP Toolset (configure connection)
3. Add it to your agent
4. Run and test the agent

**Step 1: Choose MCP Server**

For this demo, we'll use the **[Everything MCP Server](https://github.com/modelcontextprotocol/servers/tree/main/src/everything)** - an npm package (`@modelcontextprotocol/server-everything`) designed for testing MCP integrations.

It provides a `getTinyImage` tool that returns a simple test image (16x16 pixels, Base64-encoded). **Find more servers:** [modelcontextprotocol.io/examples](https://modelcontextprotocol.io/examples)

**‼️ NOTE: This is a demo server to learn MCP.** In production, you'll use servers for Google Maps, Slack, Discord, etc.

**Step 2: Create the MCP Toolset**

The `McpToolset` is used to integrate an ADK Agent with an MCP Server.

**What the code does:**
- Uses `npx` (Node package runner) to run the MCP server
- Connects to `@modelcontextprotocol/server-everything`
- Filters to only use the `getTinyImage` tool (the server has others, but we only need this one)

In [8]:
# MCP integration
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx", # running the MCP server via npx
            args=[
                "-y",
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ), 
        timeout = 30,
    )
)
print("MCP tool linked and created")

MCP tool linked and created


#### **Behind the scenes:**
1. **Server Launch**: ADK runs `npx -y @modelcontextprotocol/server-everything`
2. **Handshake**: Establishes stdio communication channel
3. **Tool Discovery**: Server tells ADK: "I provide getTinyImage" functionality
4. **Integration**: Tools appear in agent's tool list automatically
5. **Execution**: When agent calls `getTinyImage()`, ADK forwards to MCP server
6. **Response**: Server result is returned to agent seamlessly

**Why This Matters**: You get instant access to tools without writing integration code!

**Step 3: Add MCP tool to agent**

Let's add the `mcp_server` to the agent's tool array and update the agent's instructions to handle requests to generate tiny images.

In [9]:
# create an image agent with MCP integration
image_agent = LlmAgent(
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    name = "image_agent",
    instruction="Use the MCP tool to generate images for the user queries",
    tools=[mcp_image_server]
)

In [11]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=image_agent)

response = await runner.run_debug(" Provide a small image of earth", verbose=True)


 ### Created new session: debug_session_id

User >  Provide a small image of earth


/Users/mario/Desktop/kaggle/kaggle/lib/python3.13/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


image_agent > [Calling tool: getTinyImage({})]
image_agent > [Tool result: {'content': [{'type': 'text', 'text': 'This is a tiny image:'}, {'type': 'image', 'data': 'iVBORw0KG...]
image_agent > Here is a tiny image.


In [13]:
from IPython.display import display, Image as IPImage
import base64

# Loop through each response event from the agent
for event in response:
    # Check if the event has content with parts
    if event.content and event.content.parts:
        # Loop through each part of the content
        for part in event.content.parts:
            # Check if this part has a function response (tool execution result)
            if hasattr(part, "function_response") and part.function_response:
                # Get the content array from the function response
                for item in part.function_response.response.get("content", []):
                    # Check if this item is an image
                    if item.get("type") == "image":
                        # Decode the base64 image data and display it in the notebook
                        display(IPImage(data=base64.b64decode(item["data"])))

### 2.3: Extending to Other MCP Servers

The same pattern works for any MCP server - only the `connection_params` change. Here are some examples:

#### **👉 Kaggle MCP Server** - For dataset and notebook operations

Kaggle provides an MCP server that lets your agents interact with Kaggle datasets, notebooks, and competitions.

**Connection example:**
```python
McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command='npx',
            args=[
                '-y',
                'mcp-remote',
                'https://www.kaggle.com/mcp'
            ],
        ),
        timeout=30,
    )
)
```

**What it provides:**
- 📊 Search and download Kaggle datasets
- 📓 Access notebook metadata
- 🏆 Query competition information etc.,

**Learn more:** [Kaggle MCP Documentation](https://www.kaggle.com/docs/mcp)

#### **👉 GitHub MCP Server** - For PR/Issue analysis

```python
McpToolset(
    connection_params=StreamableHTTPServerParams(
        url="https://api.githubcopilot.com/mcp/",
        headers={
            "Authorization": f"Bearer {GITHUB_TOKEN}",
            "X-MCP-Toolsets": "all",
            "X-MCP-Readonly": "true"
        },
    ),
)
```

**More resources:** [ADK Third-party Tools Documentation](https://google.github.io/adk-docs/tools/third-party/)

---
## 🔄 Section 3: Long-Running Operations (Human-in-the-Loop)

So far, all tools execute and return immediately:


> ```User asks → Agent calls tool → Tool returns result → Agent responds```


**But what if your tools are long-running or you need human approval before completing an action?**

Example: A shipping agent should ask for approval before placing a large order.


> ```User asks → Agent calls tool → Tool PAUSES and asks human → Human approves → Tool completes → Agent responds```


This is called a **Long-Running Operation (LRO)** - the tool needs to pause, wait for external input (human approval), then resume.

**When to use Long-Running Operations:**

- 💰 **Financial transactions** requiring approval (transfers, purchases)
- 🗑️ **Bulk operations** (delete 1000 records - confirm first!)
- 📋 **Compliance checkpoints** (regulatory approval needed)
- 💸 **High-cost actions** (spin up 50 servers - are you sure?)
- ⚠️ **Irreversible operations** (permanently delete account)

### 3.1: What We're Building Today

Let's build a **shipping coordinator agent with one tool** that:
- Auto-approves small orders (≤5 containers)
- **Pauses and asks for approval** on large orders (>5 containers)
- Completes or cancels based on the approval decision

This demonstrates the core long-running operation pattern: **pause → wait for human input → resume**.

### 3.2: The Shipping Tool with Approval Logic

Here's the complete function.

#### The `ToolContext` Parameter

Notice the function signature includes `tool_context: ToolContext`. ADK automatically provides this object when your tool runs. It gives you two key capabilities:

1. **Request approval:** Call `tool_context.request_confirmation()`
2. **Check approval status:** Read `tool_context.tool_confirmation`